<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/PyHRV_Fitabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install glob2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from glob import glob
import os
import pandas as pd
path = "/content/drive/MyDrive/Iqram Sir/Fitabase data pulls - seperate"

In [4]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm.notebook import tqdm

def get_content(patient_folder_adrs):

  post_adrs = sorted(glob(patient_folder_adrs+"/*"))[0]
  

  pre_adrs = sorted(glob(patient_folder_adrs+"/*"))[1]

  for i in glob(post_adrs+"/*"):
    content = i.split("_")
    if "heartrate" in content and "seconds" in content:
      post_sec = i
    if "minuteMETsNarrow" in content:
      post_met = i


  for i in glob(pre_adrs+"/*"):
    content = i.split("_")
    if "heartrate" in content and "seconds" in content:
      pre_sec = i
    if "minuteMETsNarrow" in content:
      pre_met = i
 
  return post_sec,post_met, pre_sec, pre_met





def signle_sample(df_1min,df_met):
  list_1 = list(df_1min["Time"])
  list_3 = list(df_met["ActivityMinute"])


  hr = []
  time = []
  mets = []

  if len(list_1)>len(list_3):
    for i in tqdm(list_1):
      if  i in list_3:
        hr.append(list(df_1min.query("Time == '"+i+"'")['Value'])[0])
        mets.append(list(df_met.query("ActivityMinute == '"+i+"'")['METs'])[0])
        time.append(i)

  else:
    for i in tqdm(list_3):
      if i in list_1:
        hr.append(list(df_1min.query("Time == '"+i+"'")['Value'])[0])
        mets.append(list(df_met.query("ActivityMinute == '"+i+"'")['METs'])[0])
        time.append(i)

  d={
    "Time" : time,
     "HR(BPM)" : hr,
     "METs" : mets
  }

  dataFrame = pd.DataFrame.from_dict(d)
  return dataFrame


def test_signle_sample(data,df_1min,df_met):
  total_row = data.shape[0]
  print("Total DataRow: ",total_row)

  err_row=[]

  for i in tqdm(range(total_row)):
    print(data["Time"][i], data['HR(BPM)'][i],data['METs'][i])

    hr = list(df_1min.query("Time == '"+data["Time"][i]+"'")['Value'])[0]
    mets = list(df_met.query("ActivityMinute == '"+data["Time"][i]+"'")['METs'])[0]
    print(hr,mets)
    print("------------------------------------------------------------")

    if hr != data['HR(BPM)'][i] or mets != data['METs'][i]:
      err_row.append((data["Time"][i],i))
  return err_row




def range_list(k):
  list_of = []
  main_list = []

  for i in range(len(k)-1):
 
    if k[i+1]-k[i] == 1:
      list_of.append(k[i])
    else:
      list_of.append(k[i])
      main_list.append(list_of)
      list_of=[]
    if i == len(k)-2:
      if k[i+1]-k[i] == 1:
        list_of.append(k[i+1])
        main_list.append(list_of)
      else:
        main_list.append([k[i+1]])
  
  return main_list






def mets_minute_refine(data,minutes,upper_lim,lower_lim):

  mets_none = []
  mets_more = []
  for i in range(data.shape[0]):

    if lower_lim< data["METs"][i] <= upper_lim:
    #if data["METs"][i] > 10:
      mets_more.append(i)
    else:
      mets_none.append(i)

  main_l = range_list(mets_more)

  removable_list = []
  for i in range(len(main_l)-1):
    if len(main_l[i]) < minutes+1:
      for j in main_l[i]:
        removable_list.append(j)

  df1=data.drop(data.index[mets_none+removable_list]).reset_index()
  return df1





from datetime import datetime

def formate24(t):
  time = t.split(" ")
  m2 = time[1]+" "+time[2]
  in_time = datetime.strptime(m2, "%I:%M:%S %p")
  out_time = datetime.strftime(in_time, "%H:%M:%S")
  return out_time


def dif_min(start_time, end_time):
  t1 = datetime.strptime(start_time, "%H:%M:%S")
  t2 = datetime.strptime(end_time, "%H:%M:%S")

  delta = t2 - t1

  return delta.total_seconds()/60







def time_limit_check(test_data,min_limit):
  time_diff_list = []
  let = range_list(list(test_data['index']))
  for i in tqdm(let):

    start_time = formate24(list(test_data.query("index == "+str(i[0])+"")['Time'])[0])
    end_time = formate24(list(test_data.query("index == "+str(i[-1])+"")['Time'])[0])
    diff = dif_min(start_time,end_time)
    if diff < min_limit:
      time_diff_list.append(i)
  
  return time_diff_list


def single_segment_refine(single_list,min_limit):
  temp = []
  main = []
  if len(single_list)>1:
    for i in range(len(single_list)-1):
      k = formate24(list(test_data.query("index == "+str(single_list[i+1])+"")['Time'])[0])
      l = formate24(list(test_data.query("index == "+str(single_list[i])+"")['Time'])[0])
      diff = dif_min(l,k)
      #print(k,l,diff)
      if diff == 1 or diff == -1439.0:
        temp.append(single_list[i])
        flage = 1
      else:
        temp.append(single_list[i])
        if len(temp) >= min_limit+1:
          main.append(temp)
        temp=[]
        flage = 0

    if flage == 1:
      if len(temp) >= min_limit+1:
        k = formate24(list(test_data.query("index == "+str(single_list[i+1])+"")['Time'])[0])
        l = formate24(list(test_data.query("index == "+str(single_list[i])+"")['Time'])[0])
        diff = dif_min(l,k)
        if diff == 1 or diff == -1439.0:
          temp.append(single_list[i+1])
          main.append(temp)
        else:
          main.append(temp)


  return main


def final_refination(data,min_lim):
  min_lim = 5
  print("Time Limit Checking:")
  time_diff_list = time_limit_check(data,min_lim)
  print()

  removeable_list=[]

  print("Unwanted index list making:")
  for i in tqdm(time_diff_list):
  # print(i)
    lis = single_segment_refine(i,min_lim)
    combined = []

    for j in lis:
      combined=combined+j


    for k in i:
      if k not in combined:
        removeable_list.append(k)
  

  print()
  print("Refining Dataset:")

  for i in tqdm(removeable_list):
    data.drop(data.loc[data['index']==i].index, inplace=True)

  data.reset_index()

  return data





#------------------------------------------------------------------------------------------------
def remove_sec(l):
  k = datetime.strptime(l, "%m/%d/%Y %I:%M:%S %p")
  out_time = datetime.strftime(k, "%-m/%-d/%Y %-I:%M %p")
  return out_time




def add_sec(l):
  k = datetime.strptime(l, "%m/%d/%Y %I:%M %p")
  out_time = datetime.strftime(k, "%-m/%-d/%Y %-I:%M:%S %p")
  return out_time





def sec_met(df_sec,df_met):
  li_met = list(df_met['ActivityMinute'])
  li_met_s = []
  li_sec = list(df_sec['Time'])

  for i in li_met:
    li_met_s.append(remove_sec(i))



  activity_time=[]
  hr=[]
  mets=[]

  print("Finalizing The DataFrame")
  for i in tqdm(li_sec):
    if remove_sec(i) in li_met_s:
      activity_time.append(i)
      hr.append(list(df_sec.query("Time == '"+i+"'")['Value'])[0])
      mets.append(list(df_met.query("ActivityMinute == '"+add_sec(remove_sec(i))+"'")['METs'])[0])
  

  d={
    "ActivityMinute" : activity_time,
    "HR(BPM)" : hr,
    "METs" : mets
  }

  dataFrame = pd.DataFrame.from_dict(d)
  return dataFrame


def find_freq(lis):
  frequency = {}
  for item in tqdm(lis):
    if item in frequency:
        frequency[item] += 1
    else:
        frequency[item] = 1
  return frequency





def hr_range_permin(dfSec,freq):
  lis = []

  print("Removing 'Second' Section....")
  for i in tqdm(list(dfSec["Time"])):
    lis.append(remove_sec(i))

  print("Finding Frequency of Each 'Minute'....")
  dic = find_freq(lis)

  print("Creating Removable List of 'Minutes'...")
  removable_list = []
  for i in tqdm(dic):
    if dic[i] < freq:
      removable_list.append(i)


  print("Configuring HR/Min...")
  for i in tqdm(list(dfSec["Time"])):

    if remove_sec(i) in removable_list:
      dfSec.drop(dfSec[dfSec['Time'] == i].index, inplace = True)

  dfSec = dfSec.reset_index(drop=True)
  print("Process Complete..")
  return dfSec





def mets_activity(dfMets,up,low):

  dfMets.drop(dfMets[dfMets['METs'] >up].index, inplace = True)
  dfMets.drop(dfMets[dfMets['METs'] <=low].index, inplace = True)
  dfMets = dfMets.reset_index(drop = True)
  return dfMets





In [6]:
subject = '001-JV'



for i in glob(path+"/*"):
  if subject in i.split("/"):
    Post_sec,Post_met, Pre_sec,Pre_met = get_content(i)

df_post_sec = pd.read_csv(Post_sec)
df_post_met = pd.read_csv(Post_met)

df_pre_sec = pd.read_csv(Pre_sec)
df_pre_met = pd.read_csv(Pre_met)


In [23]:
#@title Select METs Ranges { run: "auto" }
option = "45 \u003C METs \u003C=60" #@param ["0 < METs <= 10","10 < METs <= 25" ,"25 < METs <= 45", "45 < METs <=60","60<METs"]
print('You selected', option)

if option == "0 \u003C METs \u003C= 10":
  upper_lim = 10
  lower_lim = 0
  activity = "Resting"
  

elif option == "10 \u003C METs \u003C= 25":
  upper_lim = 25
  lower_lim = 10
  activity = "Light_Activity"


elif option == "25 \u003C METs \u003C= 45":
  upper_lim = 45
  lower_lim = 25
  activity = "Moderate_Activity"

elif option == "45 \u003C METs \u003C=60":
  upper_lim = 60
  lower_lim = 45
  activity = "Heavy_Activity"

elif option == "60\u003CMETs":
  upper_lim = 10000
  lower_lim = 60
  activity = "Heavy_Activity"

print("Upper Limit: ",upper_lim)
print("Lower Limit: ",lower_lim)
print("Activity: ",activity)



You selected 45 < METs <=60
Upper Limit:  60
Lower Limit:  45
Activity:  Heavy_Activity


In [27]:
%%time


hr_range_per_min = 6


df_post_sec = hr_range_permin(df_post_sec,hr_range_per_min)


df_post_met = mets_activity(df_post_met,upper_lim,lower_lim)

df_post_sec.to_csv("/content/drive/MyDrive/Iqram Sir/Final_Fitabase_pyHRV/df_post_sec.csv",index = False)
df_post_met.to_csv("/content/drive/MyDrive/Iqram Sir/Final_Fitabase_pyHRV/df_post_met.csv",index = False)


d = sec_met(df_post_sec,df_post_met)
d.to_csv("/content/drive/MyDrive/Iqram Sir/Final_Fitabase_pyHRV/"+subject+"_"+str(hr_range)+"_"+activity+"_post_condition.csv",index =False)

Removing 'Second' Section....


  0%|          | 0/294630 [00:00<?, ?it/s]

Finding Frequency of Each 'Minute'....


  0%|          | 0/294630 [00:00<?, ?it/s]

Creating Removable List of 'Minutes'...


...

Configuring HR/Min...


  0%|          | 0/294630 [00:00<?, ?it/s]

Process Complete..
Finalizing The DataFrame


  0%|          | 0/270974 [00:00<?, ?it/s]

CPU times: user 20min 11s, sys: 5.69 s, total: 20min 16s
Wall time: 20min 32s


In [7]:
data = pd.read_csv("/content/drive/MyDrive/Iqram Sir/Final_Fitabase_pyHRV/001-JV_6_Heavy_Activity_post_condition.csv")

In [8]:
data.head()

,ActivityMinute,HR(BPM),METs
0,12/18/2021 12:58:08 AM,112,47
1,12/18/2021 12:58:13 AM,113,47
2,12/18/2021 12:58:18 AM,113,47
3,12/18/2021 12:58:23 AM,113,47
4,12/18/2021 12:58:28 AM,112,47


In [16]:
data.shape

(3887, 3)

In [ ]:
def formate24(t):
  #time = t.split(" ")
  #m2 = time[1]+" "+time[2]
  in_time = datetime.strptime(t, "%m/%d/%Y %I:%M:%S %p")
  out_time = datetime.strftime(in_time, "%m/%d/%Y %H:%M")
  return out_time


def dif_min(start_time, end_time):
  t1 = datetime.strptime(start_time, "%m/%d/%Y %H:%M")
  t2 = datetime.strptime(end_time, "%m/%d/%Y %H:%M")

  delta = t2 - t1

  return delta.total_seconds()/60



def range_list(k):
  list_of = []
  main_list = []

  for i in range(len(k)-1):

    if dif_min(formate24(k[i]),formate24(k[i+1])) == 1:
      list_of.append(k[i])
    else:
      list_of.append(k[i])
      main_list.append(list_of)
      list_of=[]
    if i == len(k)-2:
      if dif_min(formate24(k[i]),formate24(k[i+1])) == 1:
        list_of.append(k[i+1])
        main_list.append(list_of)
      else:
        main_list.append([k[i+1]])

  return main_list

In [88]:
li=[]

for i in list(data["ActivityMinute"]):
  li.append(i)


In [89]:
dic = find_freq(li)

  0%|          | 0/3887 [00:00<?, ?it/s]

In [ ]:
dic

In [90]:
time  = list(dic.keys())

In [20]:
dcc = {"Time": list(dic.keys())}

In [ ]:
time

In [86]:
lolo = range_list(time)
lolo

[['12/18/2021 12:58:08 AM'],
 ['12/18/2021 12:58:13 AM'],
 ['12/18/2021 12:58:18 AM'],
 ['12/18/2021 12:58:23 AM'],
 ['12/18/2021 12:58:28 AM'],
 ['12/18/2021 12:58:33 AM'],
 ['12/18/2021 12:58:39 AM'],
 ['12/18/2021 12:58:44 AM'],
 ['12/18/2021 12:58:49 AM'],
 ['12/18/2021 12:58:54 AM'],
 ['12/18/2021 12:58:59 AM'],
 ['12/19/2021 6:39:03 PM'],
 ['12/19/2021 6:39:08 PM'],
 ['12/19/2021 6:39:13 PM'],
 ['12/19/2021 6:39:18 PM'],
 ['12/19/2021 6:39:23 PM'],
 ['12/19/2021 6:39:28 PM'],
 ['12/19/2021 6:39:33 PM'],
 ['12/19/2021 6:39:38 PM'],
 ['12/19/2021 6:39:43 PM'],
 ['12/19/2021 6:39:48 PM'],
 ['12/19/2021 6:39:53 PM'],
 ['12/19/2021 6:39:58 PM'],
 ['12/22/2021 9:01:00 AM'],
 ['12/22/2021 9:01:05 AM'],
 ['12/22/2021 9:01:10 AM'],
 ['12/22/2021 9:01:15 AM'],
 ['12/22/2021 9:01:20 AM'],
 ['12/22/2021 9:01:25 AM'],
 ['12/22/2021 9:01:40 AM'],
 ['12/22/2021 9:01:45 AM'],
 ['12/22/2021 9:01:50 AM'],
 ['12/22/2021 9:01:55 AM'],
 ['12/24/2021 9:10:00 AM'],
 ['12/24/2021 9:10:05 AM'],
 ['12/24/

In [50]:
def time_limit_check(time,min_limit):
  time_diff_list = []
  
  for i in tqdm(range(len(time)-1)):

    start_time = formate24(time[i])
    end_time = formate24(time[i+1])
    diff = dif_min(start_time,end_time)
    if diff < min_limit:
      time_diff_list.append(time[i])
  
  return time_diff_list

In [60]:
lolo

'12/18/2021 12:58 AM'

In [52]:
lolo

['12/24/2021 9:10 AM',
 '12/30/2021 10:47 AM',
 '12/30/2021 10:48 AM',
 '12/30/2021 12:07 PM',
 '12/30/2021 12:08 PM',
 '12/30/2021 12:09 PM',
 '12/30/2021 12:11 PM',
 '12/30/2021 12:12 PM',
 '12/30/2021 12:13 PM',
 '12/30/2021 9:21 PM',
 '12/31/2021 9:18 AM',
 '12/31/2021 9:19 AM',
 '12/31/2021 8:59 PM',
 '12/31/2021 11:02 PM',
 '1/1/2022 10:55 AM',
 '1/4/2022 12:29 PM',
 '1/4/2022 1:28 PM',
 '1/4/2022 1:54 PM',
 '1/4/2022 1:55 PM',
 '1/4/2022 1:56 PM',
 '1/4/2022 1:57 PM',
 '1/4/2022 9:55 PM',
 '1/4/2022 9:56 PM',
 '1/5/2022 9:09 AM',
 '1/5/2022 9:13 AM',
 '1/5/2022 9:15 AM',
 '1/5/2022 9:16 AM',
 '1/5/2022 9:28 AM',
 '1/5/2022 9:29 AM',
 '1/5/2022 9:32 AM',
 '1/5/2022 9:33 AM',
 '1/5/2022 9:39 AM',
 '1/5/2022 9:40 AM',
 '1/5/2022 9:42 AM',
 '1/5/2022 9:43 AM',
 '1/5/2022 9:45 AM',
 '1/5/2022 9:46 AM',
 '1/5/2022 9:47 AM',
 '1/5/2022 10:22 AM',
 '1/5/2022 10:31 AM',
 '1/5/2022 12:15 PM',
 '1/5/2022 1:10 PM',
 '1/5/2022 3:24 PM',
 '1/5/2022 3:34 PM',
 '1/5/2022 3:38 PM',
 '1/5/2022 3:

In [42]:
start_time = formate24(time[0])
end_time = formate24(time[1]

print(start_time)
print(end_time)

print(dif_min(start_time,end_time))

12/18/2021 00:58
12/19/2021 18:39
2501.0


00:58
18:39


In [22]:
dc = pd.DataFrame(dcc)

In [26]:
dc =  dc.reset_index()

In [27]:
dc

,index,Time
0,0,12/18/2021 12:58 AM
1,1,12/19/2021 6:39 PM
2,2,12/22/2021 9:01 AM
3,3,12/24/2021 9:10 AM
4,4,12/24/2021 9:11 AM
...,...,...
380,380,1/17/2022 2:09 PM
381,381,1/17/2022 6:35 PM
382,382,1/17/2022 6:36 PM
383,383,1/18/2022 4:27 PM


In [34]:


from datetime import datetime

def formate24(t):
  time = t.split(" ")
  m2 = time[1]+" "+time[2]
  in_time = datetime.strptime(m2, "%I:%M %p")
  out_time = datetime.strftime(in_time, "%H:%M:%S")
  return out_time


def dif_min(start_time, end_time):
  t1 = datetime.strptime(start_time, "%H:%M")
  t2 = datetime.strptime(end_time, "%H:%M")

  delta = t2 - t1

  return delta.total_seconds()/60






def time_limit_check(test_data,min_limit):
  time_diff_list = []
  let = range_list(list(test_data['index']))
  for i in tqdm(let):

    start_time = formate24(list(test_data.query("index == "+str(i[0])+"")['Time'])[0])
    end_time = formate24(list(test_data.query("index == "+str(i[-1])+"")['Time'])[0])
    diff = dif_min(start_time,end_time)
    if diff < min_limit:
      time_diff_list.append(i)
  
  return time_diff_list

In [35]:
t = time_limit_check(dc,5)

  0%|          | 0/1 [00:00<?, ?it/s]

ValueError: ignored